In [1]:
import baostock as bs
import pandas as pd
import talib as ta
import matplotlib.pyplot as plt

from datetime import datetime, date

from RSI import DayRSI,WeekRSI,MonthRSI
from Stock import Stock
import dbutil
import KlineService

from IPython.core.debugger import set_trace

RSI_OVER_BUY = 80
RSI_OVER_SELL = 20
RSI_MIDDLE = 50

In [2]:

##
#  写RSI数据库
#
#
def writeRSIToDb(period, stockCode, stockName, rsi_df):
    dataList = []
    for index,rsi in rsi_df.iterrows():
        if period == "day":
            rsiObj = DayRSI(stockCode, stockName)
        elif period == "week":
            rsiObj = WeekRSI(stockCode, stockName)
        elif period == "month":
            rsiObj = MonthRSI(stockCode, stockName)
        elif period == "5m":
            rsiObj = FiveMinRSI(stockCode, stockName)
        elif period == "15m":
            rsiObj = FiftyMinRSI(stockCode, stockName)
        elif period == "30m":
            rsiObj = ThirtyMinRSI(stockCode, stockName)
        elif period == "60m":
            rsiObj = SixtyMinRSI(stockCode, stockName)

        rsiObj.date = rsi['date']
        rsiObj.rsi_6 = rsi['rsi_6']
        rsiObj.rsi_12 = rsi['rsi_12']
        rsiObj.rsi_24 = rsi['rsi_24']
        rsiObj.overBuy = rsi['overBuyFlag']
        rsiObj.overSell = rsi['overSellFlag']
        
        dataList.append(rsiObj.__dict__)
        
    mydb = dbutil.connectDB()
    collection = mydb[chooseRSICollection(period)]
    if len(dataList) > 0:
        collection.insert_many(dataList)
    else:
        raise RuntimeError("RSI数据为空")


##
#  选择不同的Kline Collection
#
def chooseRSICollection(period):
    periodRSICollection = {
        "day" : "RSI_Day",
        "week" : "RSI_Week",
        "month" : "RSI_Month",
        "5m" : "RSI_5m",
        "15m" : "RSI_15m",
        "30m" : "RSI_30m",
        "60m" : "RSI_60m"
    }
    return periodRSICollection.get(period)


def computeRSI(klineDataFrame):
    # 剔除停盘数据
    klineDataFrame = klineDataFrame[klineDataFrame['tradeStatus'] == '1']
    rsi_12days = ta.RSI(klineDataFrame['closePrice'],timeperiod=12)
    rsi_6days = ta.RSI(klineDataFrame['closePrice'],timeperiod=6)
    rsi_24days = ta.RSI(klineDataFrame['closePrice'],timeperiod=24)
    
    rsiFrame = pd.DataFrame(klineDataFrame, columns=["date"])
    rsiFrame['rsi_6'] = rsi_6days
    rsiFrame['rsi_12'] = rsi_12days
    rsiFrame['rsi_24'] = rsi_24days
    ##添加参考线位置
    rsiFrame['overBuy'] = RSI_OVER_BUY
    rsiFrame['overSell'] = RSI_OVER_SELL
    rsiFrame['middle'] = RSI_MIDDLE

    # RSI超卖和超买
    rsi_buy_position = rsiFrame['rsi_12'] > RSI_OVER_BUY
    rsi_sell_position = rsiFrame['rsi_12'] < RSI_OVER_SELL
    rsiFrame.loc[rsi_buy_position[(rsi_buy_position == True) & (rsi_buy_position.shift() == False)].index, 'overBuyFlag'] = 'Yes'
    rsiFrame.loc[rsi_sell_position[(rsi_sell_position == True) & (rsi_sell_position.shift() == False)].index, 'overSellFlag'] = 'Yes'
    return rsiFrame

##
#  计算自起始日期起的RSI
#
#
def computeAllRSIDataOfPeriod(period, startDate):
    stockDict = KlineService.allStocks()
    endDate = str(date.today())
    successCount = 0
    failCount = 0
    for key,stock in stockDict.items():
        successCount = successCount + 1
        try:
            df = KlineService.readStockKline(key, period, startDate, endDate)
            rsi_df = computeRSI(df)
            writeRSIToDb(period, key, stock["name"], rsi_df)
        except BaseException as e:
            failCount = failCount + 1
            print ("download " + key + " error:" + str(e))
        
        if successCount % 100 == 0 and successCount > 0:
            print ("download process:", successCount, " of ", len(stockDict) ," failed:", failCount)
        
    return True

In [ ]:
# downloadAllKlineDataOfSingleDay("2019-09-24")

# downloadAllKlineDataOfPeriod("day", "2017-01-01")
# downloadAllStocks("2019-09-23")
# df = allStocks()

computeAllRSIDataOfPeriod("day", "2017-01-01")

download sh.000001 error:batch op errors occurred
download sh.000002 error:batch op errors occurred
download sh.000003 error:batch op errors occurred
download sh.000004 error:batch op errors occurred
download sh.000005 error:batch op errors occurred
download sh.000006 error:batch op errors occurred
download sh.000007 error:batch op errors occurred
download sh.000008 error:batch op errors occurred
download sh.000009 error:batch op errors occurred
download sh.000010 error:batch op errors occurred
download sh.000011 error:batch op errors occurred
download sh.000012 error:batch op errors occurred
download sh.000013 error:batch op errors occurred
download sh.000015 error:batch op errors occurred
download sh.000016 error:batch op errors occurred
download sh.000017 error:batch op errors occurred
download sh.000018 error:batch op errors occurred
download sh.000019 error:batch op errors occurred
download sh.000020 error:batch op errors occurred
download sh.000021 error:batch op errors occurred
